In [6]:
from datetime import datetime
from dateutil.parser import isoparse
import sys

'''
Issue 115783 of cpython
Since Python 3.11, datetime.fromisoformat(...) accepts invalid timestamps.
The same code was tested in many Python versions via GitHub Actions:

Wrong format: 2024-01-17T15:21:00-0800
Fixed format: 2024-01-17T15:21:00-08:00
'''

# correct run
def test_correct1():
    try:
        datetime.fromisoformat("2024-01-17T15:21:00-08:00")
        # True positive
        print("test_correct1 - PASSED")
    except ValueError:
        # False negative trigger [incorrect]
        print("test_correct1 - FAILED")

def test_fails(in_stamp, i):
    try:
        datetime.fromisoformat(in_stamp)
        # False positive
        print(f"test_fail{i} - FAILED")
    except ValueError:
        # True negative trigger [incorrect]
        print(f"test_fail{i} - PASSED")


def fail_driver():
    stamps = ["2024-01-17T15:21:00-0800", "2024-01-17T15:21:00-0800", "2024-01-17T152100-08:00", \
            "20240117T15:21:00-08:00", "2024-01-17T152100-0800", \
            "20240117T15:21:00-0800", "20240117T152100-08:00"]
    i = 0
    for s in stamps:
        test_fails(s, i)
        i += 1

def test_dateutil_fails(in_stamp, i):
    try:
        isoparse(in_stamp)
        # False positive
        print(f"test_util_fail{i} - FAILED")
    except ValueError:
        # True negative trigger [incorrect]
        print(f"test_util_fail{i} - PASSED")

def util_driver():
    i = 0
    stamps = ["2024-01-17T15:21:00-0800",
            "2024-01-17T152100-08:00",
            "20240117T15:21:00-08:00",
            "2024-01-17T152100-0800",
            "20240117T15:21:00-0800",
            "20240117T152100-08:00"]
    
    for s in stamps:
        test_dateutil_fails(s, i)
        i += 1

In [7]:
# Testing
print(f"Python version {sys.version}")
test_correct1()
fail_driver()
util_driver()

Python version 3.10.11 | packaged by conda-forge | (main, May 10 2023, 19:01:19) [Clang 14.0.6 ]
test_correct1 - PASSED
test_fail0 - PASSED
test_fail1 - PASSED
test_fail2 - PASSED
test_fail3 - PASSED
test_fail4 - PASSED
test_fail5 - PASSED
test_fail6 - PASSED
test_util_fail0 - FAILED
test_util_fail1 - FAILED
test_util_fail2 - FAILED
test_util_fail3 - FAILED
test_util_fail4 - FAILED
test_util_fail5 - FAILED
